Memory Management:


Implementation for ChatBots with memory is there here: The concept of using ```RunnableWithMessageHistory``` is same there as weel.

We use the same ```RunnableWithMessageHistory``` to add in memory to the agents.
Here are the links:

All memory concepts are in section **```ChatBot```**

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

# Define tools
We first need to create the tools we want to use. 

We will use two tools: ```Tavily``` (to search online) and then a ```retriever``` over a local index we will create



# Tavily

We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1725287007, 'localtime': '2024-09-02 07:23'}, 'current': {'last_updated_epoch': 1725286500, 'last_updated': '2024-09-02 07:15', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 240, 'wind_dir': 'WSW', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 90, 'cloud': 50, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 14.7, 'windchill_f': 58.4, 'heatindex_c': 14.7, 'heatindex_f': 58.5, 'dewpoint_c': 12.2, 'dewpoint_f': 54.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 'gust_mph': 8.1, 'gust_kph': 13.0}}"},
 {'url': 'https://world-weather.info/forecast

# Retriever

We will also create a retriever over some data of our own. 




In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [4]:
retriever.invoke("how to upload a dataset")[0]

Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!', 'language': 'en'}, page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here  

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

# Create List of Tools:

Now that we have created both, we can create a list of tools that we will use downstream.



In [7]:
tools = [search, retriever_tool]

# LLM

In [8]:

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

# Bind the tools to LLM

In [10]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.



In [11]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


Now, let's try calling it with some input that would expect a tool to be called.



In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_5aGGevsbbEPC8TzTC89ddLCN', 'type': 'tool_call'}]


# Create the agent
Now that we have defined the tools and the LLM, we can create the agent. We will be using a tool calling agent - for more information on this type of agent, as well as other options, see

https://python.langchain.com/v0.2/docs/concepts/#agent_types/

If you want to see the contents of this prompt and have access to LangSmith, you can go to:

https://smith.langchain.com/hub/hwchase17/openai-functions-agent

In [13]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

# Initialize the agent

Now, we can initalize the agent with the LLM, the prompt, and the tools. 

The agent is responsible for taking in input and deciding what actions to take. 
Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). 

For more information about how to think about these components, see our conceptual guide.

# Important Note:
Note that we are passing in the ```model```, not ```model_with_tools```. That is because create_tool_calling_agent will call ```.bind_tools``` for us under the hood.

In [14]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

# Run the agent

We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions).

First up, let's how it responds when there's no need to call a tool:

In [16]:
agent_executor.invoke({"input": "hi!"})

{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the LangSmith trace

https://smith.langchain.com/public/8441812b-94ce-4832-93ec-e1114214553a/r

Let's now try it out on an example where it should be invoking the retriever



In [17]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can assist with testing in several ways:\n\n1. **Monitoring and Evaluation**: It allows developers to closely monitor and evaluate their applications, which is essential for ensuring that they function correctly under various conditions.\n\n2. **Logging Traces**: LangSmith provides multiple ways to log traces, which can be crucial for debugging and understanding how an application is performing during tests.\n\n3. **Evaluation Framework**: The platform includes an evaluation framework that allows users to define specific evaluators to compare outputs against expected results, helping to identify issues in the application logic.\n\n4. **Integration with LangChain**: While it can work independently, LangSmith can also integrate with LangChain, providing additional tools and capabilities for testing applications built using that framework.\n\nOverall, LangSmith helps developers ship their applications quickly and with

Let's take a look at the LangSmith trace to make sure it's actually calling that.

https://smith.langchain.com/public/762153f6-14d4-4c98-8659-82650f860c62/r

Now let's try one where it needs to call the search tool:



In [18]:
agent_executor.invoke({"input": "whats the weather in sf?"})

{'input': 'whats the weather in sf?',
 'output': 'The current weather in San Francisco is as follows:\n\n- **Temperature**: 17.2°C (63.0°F)\n- **Condition**: Partly cloudy\n- **Humidity**: 90%\n- **Wind**: 4.3 mph (WSW)\n- **Visibility**: 16 km (9 miles)\n\nFor more details, you can check [WeatherAPI](https://www.weatherapi.com/) or [Weather Underground](https://www.wunderground.com/hourly/us/ca/san-francisco/date/2024-09-02).'}

Let's take a look at the LangSmith trace to make sure it's actually calling that.

https://smith.langchain.com/public/36df5b1a-9a0b-4185-bae2-964e1d53c665/r

# Adding in memory to the agent

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in previous ```chat_history```. Note: it needs to be called ```chat_history``` because of the prompt we are using. If we use a different prompt, we could change the variable name


### Here we pass in an empty list of messages for chat_history because it is the first message in the chat

In [19]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})

{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

# RunnableWithMessageHistory
we wrap this in a RunnableWithMessageHistory. For more information on how to use this, see this guide.

https://python.langchain.com/v0.2/docs/how_to/message_history/



In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [24]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [25]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

# How to use memory in Agents

Because we have multiple inputs, we need to specify two things:

* ```input_messages_key```: The input key to use to add to the conversation history.
* ```history_messages_key```: The key to add the loaded messages into.

In [26]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [28]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    config={"configurable": {"session_id": session_id_1}},
)

{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [30]:
agent_with_chat_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": session_id_1}},
)

{'input': 'What is my name?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': 'Your name is Bob. How can I help you today, Bob?'}

# Very important note

This one is agent with LangChain with memory.memoryview
If you want LanGraph with Agent with memory - refer to secion Chatbots

